In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
X = tf.placeholder(tf.float32,[None,784])
X_img = tf.reshape(X,[-1,28,28,1]) # img 28x28x1 (black/white)
Y = tf.placeholder(tf.float32,[None,10])

L1 ImgIn shape = (?, 28, 28, 1)

In [ ]:
W1 = tf.Variable(tf.random.normal([3,3,1,32],stddev=0.01)) #3x3의 필터, 들어온개수, 출력개수(필터개수)
# Conv -> (?,28,28,32)
# Pool -> (?,14,14,32)
L1 = tf.nn.conv2d(X_img,W1,strides=[1,1,1,1],padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool2d(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
# Conv2D: (?,28,28,32)
# Relu: (?,28,28,32)
# MaxPool: (?,14,14,32)
# 여기까지 거치면 (?, 14, 14, 32) 임.

L2 ImgIn shape = (?, 14, 14, 32)

In [ ]:
W2 = tf.Variable(tf.random.normal([3,3,32,64],stddev=0.01)) #3x3의 필터, 들어온개수, 출력개수(필터개수)

Conv -> (?,14,14,64)<br>
Pool -> (?,7,7,64)

In [ ]:
L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool2d(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
# 여기까지 레이어 통과한 걸 Fully-Connected-Layer에 넣을 것이다.
# 그러므로 reshape 해준다.
L2 = tf.reshape(L2,[-1,7*7*64])
# Conv2D: (?,14,14,64)
# Relu: (?,14,14,64)
# MaxPool: (?,7,7,64)
# Reshape: (?,3136)
# 이렇게 feature의 개수를 확 늘였다.

In [ ]:
W3 = tf.get_variable("W2",shape=[7*7*64,10],initializer = tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random.normal([10]))
hypothesis = tf.matmul(L2,W3)+b
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print("Learning Started. It takes sometime.")

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs,Y:batch_ys}
        c,_ = sess.run([cost,optimizer],feed_dict=feed_dict)
        avg_cost +=c/total_batch
    print("Epoch:",epoch,"cost:",avg_cost)
print("Learning Finished!")

In [ ]:
print("Testing...")
correct_prediction = tf.equal(tf.argmax(hypothesis,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,dtype=tf.float32))
print("Accuracy:",sess.run(accuracy,feed_dict={X:mnist.test.images,Y:mnist.test.labels}))